<a href="https://colab.research.google.com/github/bbaloun/DynamicDataGen/blob/main/TR_PoC_DynamicDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Libraries using pip
This must be done when using Colab Notebooks, since the instance of Python is not a permanent one.  Once all of the pip installs are done, the `os._exit(00)` line should be executed to reset the kernel for you.  You will receive an error message, but this is expected.


In [ ]:
!pip install --upgrade snowflake-connector-python

In [ ]:
!pip install --upgrade pandas

In [ ]:
!pip install --upgrade snowflake-sqlalchemy

In [ ]:
import os
os._exit(00)

## Import Libraries

In [ ]:
import os
import base64
import pandas as pd
from snowflake.sqlalchemy import URL
import sqlalchemy as sa
from sys import exit
from sqlalchemy import text
import math

/usr/local/lib/python3.7/dist-packages/snowflake/connector/options.py:97: UserWarning: You have an incompatible version of 'pyarrow' installed (3.0.0), please install a version that adheres to: 'pyarrow<6.1.0,>=6.0.0; extra == "pandas"'
  _expected_pyarrow_version,


## Set your Snowflake Environments#Note: The snowPassword entered into the page should be the base64 encrypted password, not the plain text password.

In [ ]:
snowUser = ''
snowRole = ''
snowAccount = ''
snowPassword = ''
snowDatabase = ''
snowSchema = ''
snowWarehouse = ''


overwrite_bit = True

## Compile Functions

In [ ]:
def getEngine(snowUser=None, snowPassword=None, snowAccount=None, snowDatabase=None, snowSchema=None, snowWarehouse=None, snowRole=None):
    try:
        base64_bytes = snowPassword.encode('ascii')
        message_bytes = base64.b64decode(base64_bytes)
        password = message_bytes.decode('ascii')

        SnowEngine = sa.create_engine(URL(
            user=snowUser,
            password=password,
            account=snowAccount,
            database=snowDatabase,
            schema=snowSchema,
            warehouse=snowWarehouse,
            role=snowRole
        ))

        return SnowEngine

    except Exception as e:
        errorStr = 'ERROR: ' + str(e)
        return errorStr

def execSQLDataFrame(SnowEngine, sqlQuery):
    try:
        if SnowEngine is None:
            print('SnowEngine argument is required')
            exit(InvalidArgsCode)
        if sqlQuery is None:
            print('sqlQuery argument is required')
            exit(InvalidArgsCode)

        resultSet = pd.read_sql(text(sqlQuery), SnowEngine)
 
        return resultSet

    except Exception as e:
        errorStr = 'ERROR (execSQLDataFrame)' + str(e)
        print(errorStr)
        raise

def execPrimaryKeyStats():
    try:
        sqlQuery = """
            SELECT 
                table_name,
                column_name,
                data_type,
                data_type_length,
                data_type_precision,
                min_length,
                max_length,
                cardinality,
                total_cardinality,
                total_null,
                pk,
                fk,
                fk_table,
                fk_field
            FROM table_meta
            WHERE pk = True
              AND fk = False;"""
        
        resultSet = execSQLDataFrame(snowEngine, sqlQuery)
        
        return resultSet

    except Exception as e:
        errorStr = 'ERROR (execPrimaryKeyStats)' + str(e)
        print(errorStr)
        raise

def execGenPKTemp(dfPKs,debug=False):
    try:
        dfResult = pd.DataFrame()
        for i,row in dfPKs.iterrows():
            sqlQuery = """
                CREATE OR REPLACE TRANSIENT TABLE pk_""" + dfPKs.table_name[i].lower() + """_""" + dfPKs.column_name[i].lower() + """ AS """
            if dfPKs.data_type[i].upper() == 'NUMBER':
                sqlQuery += """
                SELECT
                    seq8()+1::number as pktemp,
                    seq8()+1::number as """ + dfPKs.column_name[i] + """
                FROM table(generator(rowcount => """ + str(dfPKs.cardinality[i]) + """));"""
            elif dfPKs.data_type[i].upper() == 'TEXT':
                sqlQuery += """
                WITH x AS (
                    SELECT DISTINCT randstr(uniform(""" + str(dfPKs.min_length[i]) + """,""" + str(dfPKs.max_length[i]) + """,random()),random())::varchar as """ + dfPKs.column_name[i] + """
                    FROM table(generator(rowcount => """ + str(dfPKs.cardinality[i]) + """))
                    )
                SELECT (seq8()+1)::number as pktemp,
                    """ + dfPKs.column_name[i] + """
                FROM x;"""    
            elif dfPKs.data_type[i].upper() == 'DATE':
                sqlQuery += """
                WITH x AS (
                    SELECT DISTINCT dateadd(day, -uniform(1, """ + str(dfPKs.cardinality[i]*10) + """, random()), date_trunc(day, current_date()))::date as """ + dfPKs.column_name[i] + """
                    FROM table(generator(rowcount => """ + str(dfPKs.cardinality[i]) + """))
                    )
                SELECT (seq8()+1)::number as pktemp,
                    """ + dfPKs.column_name[i] + """
                FROM x;""" 
            elif dfPKs.data_type[i].upper() == 'TIMESTAMP' or dfPKs.data_type[i].upper() == 'TIMESTAMP_LTZ' or dfPKs.data_type[i].upper() == 'TIMESTAMP_NTZ' or dfPKs.data_type[i].upper() == 'TIMESTAMP_TZ':
                sqlQuery += """
                WITH x AS (
                    SELECT DISTINCT (date_part(epoch_second, current_date()) - (uniform(1, """ + str(dfPKs.cardinality[i]*3600) + """, random())))::timestamp as """ + dfPKs.column_name[i] + """
                    FROM table(generator(rowcount => """ + str(dfPKs.cardinality[i]) + """))
                    )
                SELECT (seq8()+1)::number as pktemp,
                    """ + dfPKs.column_name[i] + """
                FROM x;""" 
            elif dfPKs.data_type[i].upper() == 'BOOLEAN':
                sqlQuery += """
                SELECT *
                FROM (VALUES (1::number,True::boolean),(2::number,False::boolean))x(pktemp,""" + dfPKs.column_name[i] + """);"""

            print(sqlQuery)

            if debug == False:
                if i==0:
                    dfResult = execSQLDataFrame(snowEngine, sqlQuery)
                else:
                    dfTemp = execSQLDataFrame(snowEngine, sqlQuery)
                    dfResult = dfResult.append(dfTemp, ignore_index = True)

        return dfResult

    except Exception as e:
        errorStr = 'ERROR (execGenPKTemp)' + str(e)
        print(errorStr)
        raise

def execGenMultiKeyTemp(dfTables,debug=False):
    try:
        dfResult = pd.DataFrame()
        for i,row in dfTables.iterrows():

            if dfTables.pk_cnt[i] > 1:
                dfPKColumns = execMultiKeyStats(dfTables.table_name[i].upper())
                sqlQuery = """
                    CREATE OR REPLACE TRANSIENT TABLE pk_multi_""" + dfTables.table_name[i].lower() + """ AS
                    """
                sqlXQuery = """
                    CREATE OR REPLACE TEMP TABLE x AS
                    SELECT DISTINCT """
                sqlYQuery = """
                    CREATE OR REPLACE TEMP TABLE y AS
                    SELECT """
                
                for j,row in dfPKColumns.iterrows():
                    sqlXQuery += """
                        uniform(1,""" + str(dfPKColumns.cardinality[j]) + """,random()) as """ + dfPKColumns.column_name[j].lower() + ""","""
                
                sqlXQuery = sqlXQuery[:-1] + """
                    FROM table(generator(rowcount => """ + str(dfPKColumns.total_cardinality[0]) + """));"""
                
                for k,row in dfPKColumns.iterrows():
                    sqlYQuery += """
                            pk""" + str(k) + """.""" + dfPKColumns.pk_field_name[k].lower() + """ as """ + dfPKColumns.column_name[k].lower() + ""","""
                
                sqlYQuery = sqlYQuery[:-1] + """ 
                    FROM x """
                
                for c,row in dfPKColumns.iterrows():
                    sqlYQuery += """
                    JOIN pk_""" + dfPKColumns.pk_table_name[c].lower() + """_""" + dfPKColumns.pk_field_name[c].lower() + """ AS pk""" + str(c) + """
                        ON x.""" + dfPKColumns.column_name[c].lower() + """ = pk""" + str(c) + """.pktemp"""
                sqlYQuery += """;"""
                
                sqlQuery += """
                    SELECT (seq8()+1)::number as pktemp,"""
                
                for p,row in dfPKColumns.iterrows():
                    sqlQuery += """
                            """ + dfPKColumns.column_name[p].lower() + ""","""
                
                sqlQuery = sqlQuery[:-1] + """
                    FROM y;"""

                print(sqlXQuery)
                print(sqlYQuery)
                print(sqlQuery)

                if debug == False:
                    dfTemp = execSQLDataFrame(snowEngine, sqlXQuery)
                    dfTemp = execSQLDataFrame(snowEngine, sqlYQuery)
                    dfTemp = execSQLDataFrame(snowEngine, sqlQuery)
                    dfResult = dfResult.append(dfTemp, ignore_index = True)

        return dfResult

    except Exception as e:
        errorStr = 'ERROR (execGenMultiKeyTemp)' + str(e)
        print(errorStr)
        raise

def execTableList():
    try:
        sqlQuery = """
        SELECT table_name, SUM(CASE WHEN pk = True THEN 1 ELSE 0 END) as pk_cnt
        FROM table_meta
        GROUP BY table_name;"""
        
        resultSet = execSQLDataFrame(snowEngine, sqlQuery)
        
        return resultSet

    except Exception as e:
        errorStr = 'ERROR (execTableList)' + str(e)
        print(errorStr)
        raise

def execMultiKeyStats(table_name):
    try:
        sqlQuery = """
            SELECT   
                table_name,
                column_name,
                cardinality,
                total_cardinality,
                total_null,
                CASE WHEN fk = False THEN table_name ELSE fk_table END as pk_table_name,
                CASE WHEn fk = False THEN column_name ELSE fk_field END as pk_field_name
            FROM table_meta
            WHERE pk = True
              AND upper(table_name) = '""" + table_name.upper() + """';"""

        resultSet = execSQLDataFrame(snowEngine, sqlQuery)

        return resultSet

    except Exception as e:
        errorStr = 'ERROR (execMultiKeyStats)' + str(e)
        print(errorStr)
        raise

def execTableStats(table_name):
    try:
        sqlQuery = """
            SELECT
                case when pk = True then count(case when pk = True then pk else null end) over (partition by table_name) else -1 end::number as pk_cnt,
                case when pk = True then dense_rank() over (partition by table_name order by pk desc, column_name) else -1 end as pk_first,
                case when pk = False and fk = True then row_number() over (partition by CASE WHEN pk=False THEN fk_table ELSE NULL END order by fk_field) else -1 end::number as fk_first, 
                case when pk = False and fk = True then count(fk_field) over (partition by CASE WHEN pk=False THEN fk_table ELSE NULL END) else -1 end::number as fk_field_cnt, 
                case when pk = False and fk = True then rank() over (partition by table_name order by CASE WHEN pk=False THEN fk_table ELSE NULL END) else -1 end as fk_rank,
                table_name,
                column_name,
                data_type,
                data_type_length,
                data_type_precision,
                min_length,
                max_length,
                cardinality,
                total_cardinality,
                total_null,
                pk,
                fk,
                fk_table,
                fk_field
            FROM table_meta
            WHERE upper(table_name) = '""" + table_name.upper() + """'
            order by nullif(pk_cnt,-1), nullif(pk_first,-1), nullif(fk_first,-1), nullif(fk_rank,-1);"""
        
        resultSet = execSQLDataFrame(snowEngine, sqlQuery)
        
        return resultSet

    except Exception as e:
        errorStr = 'ERROR (execTableStats)' + str(e)
        print(errorStr)
        raise

def execGenColumnSQL(dfStats,i):
    try:
        ff = ''
        if dfStats.data_type[i].upper() == 'DATE':
            fb = 'dateadd(day, -uniform(1, ' + str(dfStats.cardinality[i]) + ', random()), date_trunc(day, current_date())) '
            fe = '::date '
        elif dfStats.data_type[i].upper() == 'TIMESTAMP' or dfStats.data_type[i].upper() == 'TIMESTAMP_LTZ' or dfStats.data_type[i].upper() == 'TIMESTAMP_NTZ':
            fb = '(date_part(epoch_second, current_date()) - (uniform(1, ' + str(dfStats.cardinality[i]) + ', random()))) '
            fe = '::timestamp '
        elif dfStats.data_type[i].upper() == 'TEXT':
            ff = 'uniform(1,' + str(dfStats.cardinality[i]) + ',random()) as seed_' + dfStats.column_name[i].lower() + ', '
            fb = 'randstr(seed_' + dfStats.column_name[i].lower() + '*' + str(dfStats.max_length[i]) + '/' + str(dfStats.cardinality[i]) + ',seed_' + dfStats.column_name[i].lower() + ') '
            #randstr(uniform(' + str(dfStats.min_length[i]) + ',' + str(dfStats.max_length[i]) + ', random()),uniform(1,'+ str(dfStats.cardinality[i]) + ',random())) '
            fe = '::varchar(' + str(int(dfStats.data_type_length[i])) + ') '
        elif dfStats.data_type[i].upper() == 'NUMBER':
            if dfStats.cardinality[i] == dfStats.total_cardinality[i] and int(dfStats.data_type_precision[i]) == 0:
                fb = '(seq8()+1)'
            elif int(dfStats.data_type_precision[i]) == 0:
                fb = 'uniform(1,' + str(dfStats.cardinality[i]) + ' , random()) '            
            else:
                fb = '(uniform(1,' + str(dfStats.cardinality[i]) + ' , random())/1000) '
            fe = '::number(' + str(int(dfStats.data_type_length[i])) + ',' + str(int(dfStats.data_type_precision[i])) + ') '
        elif dfStats.data_type[i].upper() == 'FLOAT':
            fb = "uniform(1,ceil(" + str(int(dfStats.cardinality[i])/4) + ") , random()) || '.' || uniform(1,9999,random()) "
            fe = '::float'
        else:
            fb = 'NULL '
            fe = '::' + dfStats.data_type[i].upper()

        if (fb == ''):
            print( "WARNING: Line: {0} Unknown Datatype: {1}".format(i,dfStats.data_type[i]))
        else:
            if (int(dfStats.cardinality[i]) == 0):
                formula = 'null' + fe
            elif (dfStats.total_null[i] == 0):
                formula = ff + fb + fe
            else:
                formula = ff + '(case when uniform(1,1000,random()) <= (' + str(int(dfStats.total_null[i])) + '/' + str(dfStats.total_cardinality[i]) + ')*1000 then null else ' + fb +' end) ' + fe

        formula += """ as """ + dfStats.column_name[i] + """ """

        return formula

    except Exception as e:
        errorStr = 'ERROR (execGenColumnSQL)' + str(e)
        print(errorStr)
        raise

def execInsertFinalData(dfTables,debug=False):
    try:
        dfResult = pd.DataFrame()
        for i,row in dfTables.iterrows():
            dfStats = execTableStats(str(dfTables.table_name[i]))

            strSQLx = """
            CREATE OR REPLACE TEMP TABLE x AS
            SELECT """

            for j,row in dfStats.iterrows():
                if dfStats.pk_cnt[j] > 0 and j == 0:
                    strSQLx += """ seq8()+1::number as pktemp, """
                elif dfStats.fk_rank[j] > 0 and dfStats.fk_first[j] == 1:
                    strSQLx += """ uniform(1,""" + str(dfStats.cardinality[j]) + """, random()) as fk_""" + str(dfStats.fk_rank[j]) + ""","""
                else:
                    dfStatsRow = dfStats.iloc[[j]]
                    sqlQuery = execGenColumnSQL(dfStatsRow,j)
                    strSQLx += sqlQuery + ""","""
            
            strSQLx = strSQLx[:-1] + """
                FROM table(generator(rowcount=>""" + str(dfStats.total_cardinality[0]) + """));"""

            if overwrite_bit == True:
                strSQL = """
            INSERT OVERWRITE INTO """ + dfTables.table_name[i] + """ ("""
            else:
                strSQL = """
            INSERT INTO """ + dfTables.table_name[i] + """ ("""

            for q,row in dfStats.iterrows():
                strSQL += dfStats.column_name[q] + ""","""
            strSQL = strSQL[:-1] + """
            )"""
            
            strSQL += """
                SELECT """

            for k,row in dfStats.iterrows():
                if dfStats.pk[k] == True and dfStats.pk_cnt[k] == 1 and dfStats.fk[k] == True:
                    strSQL += """ pk.""" + dfStats.fk_field[k] + ""","""
                elif dfStats.pk[k] == True:
                    strSQL += """ pk.""" + dfStats.column_name[k] + ""","""
                elif dfStats.fk_rank[k] > 0:
                    strSQL += """ fk""" + str(dfStats.fk_rank[k]) + """.""" +  dfStats.fk_field[k] + """ as """ + dfStats.column_name[k] + ""","""
                else:
                    strSQL += """ x.""" + dfStats.column_name[k] + ""","""
            
            strSQL = strSQL[:-1] + """
            FROM x"""

            for c,row in dfStats.iterrows():
                if dfStats.pk_cnt[c] == 1 and dfStats.fk[c] == False:
                    strSQL += """
            JOIN pk_""" + dfStats.table_name[c] + """_""" + dfStats.column_name[c] + """ pk
                ON x.pktemp = pk.pktemp"""
                if dfStats.pk_cnt[c] == 1 and dfStats.fk[c] == True:
                    strSQL += """
            JOIN pk_""" + dfStats.fk_table[c] + """_""" + dfStats.fk_field[c] + """ pk
                ON x.pktemp = pk.pktemp"""
                elif dfStats.pk_cnt[c] > 1 and dfStats.pk_first[c] == 1:
                        strSQL += """
            JOIN pk_multi_""" + dfStats.table_name[c] + """ pk
                ON x.pktemp = pk.pktemp"""               
                elif dfStats.fk_field_cnt[c] == 1:
                    strSQL += """
            JOIN pk_""" + dfStats.fk_table[c] + """_""" + dfStats.fk_field[c] + """ fk""" + str(dfStats.fk_rank[c]) + """
                ON x.fk_""" + str(dfStats.fk_rank[c]) + """ = fk""" + str(dfStats.fk_rank[c]) + """.pktemp"""
                elif dfStats.fk_field_cnt[c] > 1 and dfStats.fk_first[c] == 1:
                    strSQL += """
            JOIN pk_multi_""" + dfStats.fk_table[c] + """ fk""" + str(dfStats.fk_rank[c]) + """
                ON x.fk_""" + str(dfStats.fk_rank[c]) + """ = fk""" + str(dfStats.fk_rank[c]) + """.pktemp"""


            strSQL += """;"""
            
            print(strSQLx)
            print(strSQL)

            if debug == False:
                dfTemp = execSQLDataFrame(snowEngine, strSQLx)
                dfResult = dfResult.append(dfTemp, ignore_index = True)
                dfTemp = execSQLDataFrame(snowEngine, strSQL)
                dfResult = dfResult.append(dfTemp, ignore_index = True)

        return dfResult

    except Exception as e:
        errorStr = 'ERROR (execInsertFinalData)' + str(e)
        print(errorStr)
        raise

## Test Connection

In [ ]:
snowEngine = getEngine(snowUser=snowUser, snowPassword=snowPassword, snowAccount=snowAccount, snowDatabase=snowDatabase, snowSchema=snowSchema, snowWarehouse=snowWarehouse, snowRole=snowRole)

sqlQuery = 'SELECT CURRENT_VERSION();'

dfVersion = execSQLDataFrame(snowEngine, sqlQuery)

dfVersion

/usr/local/lib/python3.7/dist-packages/pandas/io/sql.py:1424: SAWarning: Dialect snowflake:snowflake will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  return self.connectable.execution_options().execute(*args, **kwargs)


,CURRENT_VERSION()
0,6.2.1


## Generate Primary Keys

In [ ]:
dfPKs = execPrimaryKeyStats()
dfPKs

,table_name,column_name,data_type,data_type_length,data_type_precision,min_length,max_length,cardinality,total_cardinality,total_null,pk,fk,fk_table,fk_field
0,DIM_OOID,OOID,TEXT,16,None,6,16,127566,127566,0,True,False,None,None
1,CLIENT_MASTER,COMPANY_CODE,TEXT,64,None,1,64,652842,3200433,1,True,False,None,None
2,CLIENT_MASTER,REGION_CODE,TEXT,4,None,4,4,120,3200433,0,True,False,None,None
3,CLIENT_MASTER,PRODUCT_CODE,TEXT,2,None,2,2,139,3200433,0,True,False,None,None


In [ ]:
dfResult = execGenPKTemp(dfPKs,debug=True)
dfResult


                CREATE OR REPLACE TRANSIENT TABLE pk_dim_ooid_ooid AS 
                WITH x AS (
                    SELECT DISTINCT randstr(uniform(6,16,random()),random())::varchar as OOID
                    FROM table(generator(rowcount => 127566))
                    )
                SELECT (seq8()+1)::number as pktemp,
                    OOID
                FROM x;

                CREATE OR REPLACE TRANSIENT TABLE pk_client_master_company_code AS 
                WITH x AS (
                    SELECT DISTINCT randstr(uniform(1,64,random()),random())::varchar as COMPANY_CODE
                    FROM table(generator(rowcount => 652842))
                    )
                SELECT (seq8()+1)::number as pktemp,
                    COMPANY_CODE
                FROM x;

                CREATE OR REPLACE TRANSIENT TABLE pk_client_master_region_code AS 
                WITH x AS (
                    SELECT DISTINCT randstr(uniform(4,4,random()),random())::varchar as REGION_CODE

""


In [ ]:
dfTables = execTableList()
dfTables

,table_name,pk_cnt
0,DIM_OOID,1
1,AP_EARNINGS_DETAILS,3
2,CLIENT_MASTER,3


In [ ]:
dfResult = execGenMultiKeyTemp(dfTables,debug=True)
dfResult


                    CREATE OR REPLACE TEMP TABLE x AS
                    SELECT DISTINCT 
                        uniform(1,91673,random()) as ooid,
                        uniform(1,40730817,random()) as aoid,
                        uniform(1,12,random()) as yyyymm
                    FROM table(generator(rowcount => 342486895));

                    CREATE OR REPLACE TEMP TABLE y AS
                    SELECT 
                            pk0.ooid as ooid,
                            pk1.aoid as aoid,
                            pk2.yyyymm as yyyymm 
                    FROM x 
                    JOIN pk_dim_ooid_ooid AS pk0
                        ON x.ooid = pk0.pktemp
                    JOIN pk_dim_aoid_aoid AS pk1
                        ON x.aoid = pk1.pktemp
                    JOIN pk_dim_date_yyyymm AS pk2
                        ON x.yyyymm = pk2.pktemp;

                    CREATE OR REPLACE TRANSIENT TABLE pk_multi_ap_earnings_details AS
                    
          

""


## Build Final Tables

In [ ]:
dfResult = execInsertFinalData(dfTables,debug=True)
dfResult


            CREATE OR REPLACE TEMP TABLE x AS
            SELECT  seq8()+1::number as pktemp,
                FROM table(generator(rowcount=>127566));

            INSERT OVERWRITE INTO DIM_OOID (OOID
            )
                SELECT  pk.OOID
            FROM x
            JOIN pk_DIM_OOID_OOID pk
                ON x.pktemp = pk.pktemp;

            CREATE OR REPLACE TEMP TABLE x AS
            SELECT  seq8()+1::number as pktemp, uniform(1,91673,random()) as seed_ooid, randstr(seed_ooid*16/91673,seed_ooid) ::varchar(16)  as OOID ,uniform(1,12,random()) as seed_yyyymm, randstr(seed_yyyymm*6/12,seed_yyyymm) ::varchar(6)  as YYYYMM ,(case when uniform(1,1000,random()) <= (142031511/342486895)*1000 then null else uniform(1,ceil(567949.0) , random()) || '.' || uniform(1,9999,random())  end) ::float as REGULAR_PAY_ ,(case when uniform(1,1000,random()) <= (209533141/342486895)*1000 then null else uniform(1,ceil(169020.75) , random()) || '.' || uniform(1,9999,random())  end) ::float as T

""


## MANUAL EXECUTE A QUERY

In [ ]:
sqlQuery = """
                CREATE OR REPLACE TEMP TABLE pk_dim_date_informationdate AS 
                SELECT
                    seq8()+1::number as pktemp,
                    TO_VARCHAR(dateadd(d,-1,(dateadd('month',-seq8(),date_trunc('month', current_date())))),'YYYYMMDD')::number as INFORMATIONDATE
                FROM table(generator(rowcount => 60));
"""
dfResults = execSQLDataFrame(snowEngine, sqlQuery)
dfResults

,status
0,Table PK_DIM_DATE_INFORMATIONDATE successfully...


In [ ]:
sqlQuery = """
    SELECT CURRENT_SCHEMA();
"""

dfResults = execSQLDataFrame(snowEngine, sqlQuery)
dfResults

,CURRENT_SCHEMA()
0,DYNAMIC_TEST_DATA_GENERATOR


In [ ]:
dfStats = execTableStats('REF_MNGD_SEG_DIM')
dfStats

,pk_cnt,pk_first,fk_first,fk_field_cnt,fk_rank,table_name,column_name,data_type,data_type_length,data_type_precision,min_length,max_length,cardinality,total_cardinality,total_null,pk,fk,fk_table,fk_field
